# 02. Data Cleaning | تنظيف البيانات

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Example 1: Data Loading and Exploration** - You need to know how to identify data quality issues first!
- ✅ **Basic pandas knowledge**: DataFrames, indexing, filtering
- ✅ **Understanding of data quality**: What are missing values, duplicates, outliers?

**If you haven't completed these**, you might struggle with:
- Understanding why data cleaning is necessary
- Knowing which cleaning method to use
- Understanding the impact of cleaning on your data

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is the SECOND example** - it fixes the problems we found in Example 1!

**Why this example SECOND?**
- **Before** you can preprocess data, you need to clean it
- **Before** you can build models, you need clean data
- **Before** you can make predictions, you need to fix data quality issues

**Builds on**: 
- 📓 Example 1: Data Loading and Exploration (we found the problems, now we fix them!)

**Leads to**: 
- 📓 Example 3: Data Preprocessing (needs clean data to work with)
- 📓 Example 4: Linear Regression (needs clean, preprocessed data)
- 📓 All ML models (all need clean data!)

**Why this order?**
1. Data cleaning fixes **quality issues** (needed before preprocessing)
2. Data cleaning teaches you **when to remove vs. impute** (critical decision-making)
3. Data cleaning shows you **the impact of outliers** (affects model accuracy)

---

## The Story: Cleaning Before Cooking | القصة: التنظيف قبل الطبخ

Imagine you're cooking a meal. **Before** you can cook, you need to clean your ingredients - remove spoiled items, wash vegetables, check for foreign objects. **After** cleaning everything, you can prepare a safe, delicious meal!

Same with machine learning: **Before** building models, we clean our data - remove duplicates, handle missing values, fix outliers. **After** cleaning, we can build accurate, reliable models!

---

## Why Data Cleaning Matters | لماذا يهم تنظيف البيانات؟

Data cleaning is essential for accurate models:
- **Missing Values**: Break ML algorithms - must be handled
- **Duplicates**: Bias your models (same data counted twice)
- **Outliers**: Skew predictions and statistics
- **Wrong Data Types**: Cause errors in calculations
- **Dirty Data = Bad Models**: No amount of ML can fix fundamentally bad data

## Learning Objectives | أهداف التعلم
1. Handle missing values (remove or impute)
2. Remove duplicate rows
3. Detect and handle outliers
4. Convert data types correctly
5. Understand trade-offs between different cleaning methods
6. Know when to remove vs. when to fix data

In [1]:
# Step 1: Import necessary libraries
# These libraries help us clean and analyze data

import pandas as pd  # For data manipulation (cleaning operations)
import numpy as np   # For numerical operations (handling NaN, calculations)
import matplotlib.pyplot as plt  # For visualizations (seeing outliers)
import seaborn as sns  # For statistical plots (data quality visualization)

print("✅ Libraries imported successfully!")
print("\n📚 What each library does:")
print("   - pandas: Clean data (remove, fill, filter)")
print("   - numpy: Handle missing values (NaN operations)")
print("   - matplotlib: Visualize data quality issues")
print("   - seaborn: Create beautiful quality check plots")

✅ Libraries imported successfully!

📚 What each library does:
   - pandas: Clean data (remove, fill, filter)
   - numpy: Handle missing values (NaN operations)
   - matplotlib: Visualize data quality issues
   - seaborn: Create beautiful quality check plots


## Part 1: Setting the Scene | الجزء الأول: إعداد المشهد

**BEFORE**: We explored our data in Example 1 and found problems - missing values, duplicates, outliers.

**AFTER**: We'll clean the data by fixing all these issues, making it ready for preprocessing and modeling!

**Why this matters**: Dirty data produces unreliable models. Cleaning is non-negotiable for good ML results!

In [2]:
# Create sample data with common data quality issues
# In real projects, you'd load this from a file and discover these issues during exploration

print("\n1. Creating sample data with common issues...")
print("إنشاء بيانات نموذجية بمشاكل شائعة...")

np.random.seed(42)  # For reproducibility
data = {
    'id': range(1, 21),
    'name': [f'Person_{i}' for i in range(1, 21)],
    'age': [25, 30, None, 35, 40, None, 28, 32, 45, 50,
            22, None, 38, 42, 29, 33, 48, None, 27, 31],  # Missing values (None = NaN)
    'salary': [50000, 60000, 55000, 70000, 80000, 65000, 58000,
               72000, 90000, 95000, 52000, 68000, 75000, 85000,
               57000, 73000, 92000, 62000, 54000, 71000],
    'department': ['IT', 'HR', 'IT', 'Finance', 'IT', 'HR', 'IT',
                   'Finance', 'IT', 'HR', 'IT', None, 'Finance', 'IT',
                   'HR', 'IT', 'Finance', 'IT', None, 'HR'],  # Missing values
    'experience_years': [2, 5, 3, 8, 10, 4, 2.5, 6, 12, 15,
                         1, 5.5, 9, 11, 3, 7, 13, 4.5, 2, 6]
}
df = pd.DataFrame(data)

print("✅ Sample data created with intentional issues:")
print("   - Missing values in 'age' and 'department'")
print("   - We'll add duplicates and outliers next")



1. Creating sample data with common issues...
إنشاء بيانات نموذجية بمشاكل شائعة...
✅ Sample data created with intentional issues:
   - Missing values in 'age' and 'department'
   - We'll add duplicates and outliers next


## Step 1: Create Sample Data with Issues | الخطوة 1: إنشاء بيانات نموذجية بمشاكل

**BEFORE**: We need to learn cleaning techniques, but we need "dirty" data to practice on.

**AFTER**: We'll create sample data with common real-world problems (missing values, duplicates, outliers) so we can practice cleaning!

**Why create dirty data?** Real datasets have these problems! We need to learn how to handle them.

In [3]:
# Add some duplicate rows to demonstrate duplicate removal
# Why add duplicates? Real data often has duplicates from data entry errors or merging issues
df = pd.concat([df, df.iloc[[0, 1]]], ignore_index=True)
print("   - Added 2 duplicate rows")

   - Added 2 duplicate rows


In [4]:
# Add some outliers to demonstrate outlier handling
# Why add outliers? Real data has outliers from errors (typos, measurement mistakes) or rare events
df.loc[18, 'salary'] = 500000  # Extreme outlier (10x normal salary - likely a typo)
df.loc[19, 'age'] = 150  # Impossible value (no one lives to 150 - data entry error)

print("\n📊 Original Data Shape:", df.shape)
print("شكل البيانات الأصلية:", df.shape)
print("   - Added 1 extreme salary outlier")
print("   - Added 1 impossible age value")
print("\n📄 Original Data (first 10 rows):")
print(df.head(10))


📊 Original Data Shape: (22, 6)
شكل البيانات الأصلية: (22, 6)
   - Added 1 extreme salary outlier
   - Added 1 impossible age value

📄 Original Data (first 10 rows):
   id       name   age  salary department  experience_years
0   1   Person_1  25.0   50000         IT               2.0
1   2   Person_2  30.0   60000         HR               5.0
2   3   Person_3   NaN   55000         IT               3.0
3   4   Person_4  35.0   70000    Finance               8.0
4   5   Person_5  40.0   80000         IT              10.0
5   6   Person_6   NaN   65000         HR               4.0
6   7   Person_7  28.0   58000         IT               2.5
7   8   Person_8  32.0   72000    Finance               6.0
8   9   Person_9  45.0   90000         IT              12.0
9  10  Person_10  50.0   95000         HR              15.0


In [5]:
# First, let's see how many missing values we have
print("\n" + "=" * 60)
print("2. Handling Missing Values")
print("معالجة القيم المفقودة")
print("=" * 60)

print("\n🔍 Missing values before cleaning:")
print("القيم المفقودة قبل التنظيف:")
missing_before = df.isnull().sum()
print(missing_before)
print(f"\n   Total missing values: {missing_before.sum()}")
print(f"   Percentage missing: {(missing_before.sum() / (df.shape[0] * df.shape[1]) * 100):.1f}%")



2. Handling Missing Values
معالجة القيم المفقودة

🔍 Missing values before cleaning:
القيم المفقودة قبل التنظيف:
id                  0
name                0
age                 4
salary              0
department          2
experience_years    0
dtype: int64

   Total missing values: 6
   Percentage missing: 4.5%


In [6]:
# Method 1: Remove rows with missing values
# .dropna() removes any row that has at least one missing value
# Why use this? If missing values are rare, it's better to remove than guess

print("\n--- Method 1: Remove rows with missing values ---")
print("--- الطريقة 1: حذف الصفوف ذات القيم المفقودة ---")
df_removed = df.dropna()
rows_removed = df.shape[0] - df_removed.shape[0]
print(f"✅ Rows after removal: {df_removed.shape[0]} (removed {rows_removed} rows)")
print(f"الصفوف بعد الحذف: {df_removed.shape[0]} (تم حذف {rows_removed} صف)")
print(f"   Data loss: {(rows_removed / df.shape[0] * 100):.1f}%")



--- Method 1: Remove rows with missing values ---
--- الطريقة 1: حذف الصفوف ذات القيم المفقودة ---
✅ Rows after removal: 17 (removed 5 rows)
الصفوف بعد الحذف: 17 (تم حذف 5 صف)
   Data loss: 22.7%


## Step 2: Handling Missing Values | الخطوة 2: معالجة القيم المفقودة

**BEFORE**: We have missing values (NaN/None) that will break our ML models.

**AFTER**: We'll either remove rows with missing values OR fill them with reasonable estimates!

**Why handle missing values?** 
- ML algorithms cannot work with missing data
- Missing values indicate incomplete information
- We must decide: **Remove** (if few missing) or **Impute** (if many missing)

**Two main strategies:**
1. **Remove**: Drop rows/columns with missing values (good if <5% missing)
2. **Impute**: Fill missing values with mean/median/mode (good if >5% missing)

In [7]:
# Method 2: Fill missing values (imputation)
# We'll use a copy so we don't modify the original
print("\n--- Method 2: Fill missing values ---")
print("--- الطريقة 2: ملء القيم المفقودة ---")
df_filled = df.copy()



--- Method 2: Fill missing values ---
--- الطريقة 2: ملء القيم المفقودة ---


In [8]:
# Method 1: Remove rows with missing values (if few missing values)
# الطريقة 1: حذف الصفوف ذات القيم المفقودة (إذا كانت قليلة)
print("\n--- Method 1: Remove rows with missing values ---")
print("--- الطريقة 1: حذف الصفوف ذات القيم المفقودة ---")
df_removed = df.dropna()
print(f"Rows after removal: {df_removed.shape[0]} (removed {df.shape[0] - df_removed.shape[0]} rows)")
print(f"الصفوف بعد الحذف: {df_removed.shape[0]} (تم حذف {df.shape[0] - df_removed.shape[0]} صف)")


--- Method 1: Remove rows with missing values ---
--- الطريقة 1: حذف الصفوف ذات القيم المفقودة ---
Rows after removal: 17 (removed 5 rows)
الصفوف بعد الحذف: 17 (تم حذف 5 صف)


In [9]:
# Method 2: Fill missing values (imputation)
# الطريقة 2: ملء القيم المفقودة (الاستبدال)
print("\n--- Method 2: Fill missing values ---")
print("--- الطريقة 2: ملء القيم المفقودة ---")
df_filled = df.copy()


--- Method 2: Fill missing values ---
--- الطريقة 2: ملء القيم المفقودة ---


In [10]:
# Fill numeric columns with mean
# ملء الأعمدة الرقمية بالمتوسط
df_filled['age'].fillna(df_filled['age'].mean(), inplace=True)

In [11]:
# Check for and remove duplicate rows
# .duplicated() finds rows that are exact duplicates
# .drop_duplicates() removes them, keeping the first occurrence

print("\n" + "=" * 60)
print("3. Removing Duplicates")
print("إزالة التكرارات")
print("=" * 60)

num_duplicates = df_filled.duplicated().sum()
print(f"\n🔍 Number of duplicates: {num_duplicates}")
print(f"عدد التكرارات: {num_duplicates}")

df_no_duplicates = df_filled.drop_duplicates()
print(f"\n✅ Rows after removing duplicates: {df_no_duplicates.shape[0]}")
print(f"الصفوف بعد إزالة التكرارات: {df_no_duplicates.shape[0]}")
print(f"   Removed {num_duplicates} duplicate row(s)")



3. Removing Duplicates
إزالة التكرارات

🔍 Number of duplicates: 2
عدد التكرارات: 2

✅ Rows after removing duplicates: 20
الصفوف بعد إزالة التكرارات: 20
   Removed 2 duplicate row(s)


In [12]:
# Fill categorical columns with mode
# ملء الأعمدة الفئوية بالقيمة الأكثر تكراراً
df_filled['department'].fillna(df_filled['department'].mode()[0], inplace=True)
print("\nMissing values after filling:")
print("القيم المفقودة بعد الملء:")
print(df_filled.isnull().sum())


Missing values after filling:
القيم المفقودة بعد الملء:
id                  0
name                0
age                 0
salary              0
department          0
experience_years    0
dtype: int64


In [13]:
print("\n" + "=" * 60)
print("4. Handling Outliers")
print("معالجة القيم الشاذة")
print("=" * 60)



4. Handling Outliers
معالجة القيم الشاذة


In [14]:
# 3. Removing Duplicates
# إزالة التكرارات
print("\n" + "=" * 60)
print("3. Removing Duplicates")
print("إزالة التكرارات")
print("=" * 60)
print(f"\nNumber of duplicates: {df_filled.duplicated().sum()}")
print(f"عدد التكرارات: {df_filled.duplicated().sum()}")
df_no_duplicates = df_filled.drop_duplicates()
print(f"Rows after removing duplicates: {df_no_duplicates.shape[0]}")
print(f"الصفوف بعد إزالة التكرارات: {df_no_duplicates.shape[0]}")


3. Removing Duplicates
إزالة التكرارات

Number of duplicates: 2
عدد التكرارات: 2
Rows after removing duplicates: 20
الصفوف بعد إزالة التكرارات: 20


In [15]:
# IQR (Interquartile Range) Method for outlier detection
# This is a statistical method that identifies values far from the median

print("\n--- IQR Method for Outlier Detection ---")
print("--- طريقة المدى الربيعي للكشف عن القيم الشاذة ---")

def detect_outliers_iqr(series):
    """
    Detect outliers using IQR method.
    Returns True for outliers, False for normal values.
    """
    Q1 = series.quantile(0.25)  # 25th percentile
    Q3 = series.quantile(0.75)  # 75th percentile
    IQR = Q3 - Q1  # Interquartile Range
    lower_bound = Q1 - 1.5 * IQR  # Lower fence
    upper_bound = Q3 + 1.5 * IQR  # Upper fence
    
    # Values outside the fences are outliers
    return (series < lower_bound) | (series > upper_bound)

print("   ✅ IQR function defined")
print("   This will identify values that are too far from the median")



--- IQR Method for Outlier Detection ---
--- طريقة المدى الربيعي للكشف عن القيم الشاذة ---
   ✅ IQR function defined
   This will identify values that are too far from the median


In [16]:
# 4. Handling Outliers
# معالجة القيم الشاذة
print("\n" + "=" * 60)
print("4. Handling Outliers")
print("معالجة القيم الشاذة")
print("=" * 60)


4. Handling Outliers
معالجة القيم الشاذة


In [17]:
# Method 1: IQR (Interquartile Range) Method
# الطريقة 1: طريقة المدى الربيعي
print("\n--- IQR Method for Outlier Detection ---")
print("--- طريقة المدى الربيعي للكشف عن القيم الشاذة ---")
def detect_outliers_iqr(series):
    """Detect outliers using IQR method"""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (series < lower_bound) | (series > upper_bound)


--- IQR Method for Outlier Detection ---
--- طريقة المدى الربيعي للكشف عن القيم الشاذة ---


In [18]:
# Check for outliers in salary
print("\nOutliers in salary column:")
print("القيم الشاذة في عمود الراتب:")
salary_outliers = detect_outliers_iqr(df_no_duplicates['salary'])
print(f"Number of outliers: {salary_outliers.sum()}")
print(df_no_duplicates[salary_outliers][['name', 'salary']])


Outliers in salary column:
القيم الشاذة في عمود الراتب:
Number of outliers: 1
         name  salary
18  Person_19  500000


In [19]:
# Remove outliers
df_clean = df_no_duplicates[~salary_outliers].copy()

In [20]:
print("\n" + "=" * 60)
print("5. Data Type Conversion")
print("تحويل أنواع البيانات")
print("=" * 60)

print("\n📊 Data types before conversion:")
print("أنواع البيانات قبل التحويل:")
print(df_clean.dtypes)



5. Data Type Conversion
تحويل أنواع البيانات

📊 Data types before conversion:
أنواع البيانات قبل التحويل:
id                    int64
name                 object
age                 float64
salary                int64
department           object
experience_years    float64
dtype: object


In [21]:
# Also remove impossible age values
df_clean = df_clean[df_clean['age'] <= 100].copy()
print(f"\nRows after removing outliers: {df_clean.shape[0]}")
print(f"الصفوف بعد إزالة القيم الشاذة: {df_clean.shape[0]}")


Rows after removing outliers: 18
الصفوف بعد إزالة القيم الشاذة: 18


In [22]:
# 5. Data Type Conversion
# تحويل أنواع البيانات
print("\n" + "=" * 60)
print("5. Data Type Conversion")
print("تحويل أنواع البيانات")
print("=" * 60)
print("\nData types before conversion:")
print("أنواع البيانات قبل التحويل:")
print(df_clean.dtypes)


5. Data Type Conversion
تحويل أنواع البيانات

Data types before conversion:
أنواع البيانات قبل التحويل:
id                    int64
name                 object
age                 float64
salary                int64
department           object
experience_years    float64
dtype: object


In [23]:
# Convert experience_years to int (rounding)
df_clean['experience_years'] = df_clean['experience_years'].round().astype(int)

In [24]:
# Final summary of cleaning process
print("\n" + "=" * 60)
print("6. Cleaning Summary")
print("ملخص التنظيف")
print("=" * 60)

original_rows = df.shape[0]
final_rows = df_clean.shape[0]
rows_removed = original_rows - final_rows

print(f"\n📊 Original rows: {original_rows}")
print(f"الصفوف الأصلية: {original_rows}")
print(f"✅ Final cleaned rows: {final_rows}")
print(f"الصفوف النهائية بعد التنظيف: {final_rows}")
print(f"\n🗑️  Rows removed: {rows_removed} ({(rows_removed/original_rows*100):.1f}%)")
print(f"الصفوف المحذوفة: {rows_removed}")

print("\n📄 Cleaned Data (first 10 rows):")
print("البيانات النظيفة:")
print(df_clean.head(10))

print("\n" + "=" * 60)
print("✅ Example 2 Complete! ✓")
print("اكتمل المثال 2! ✓")
print("=" * 60)
print("\n🎓 What you accomplished:")
print("   ✅ Handled missing values (imputed with mean/mode)")
print("   ✅ Removed duplicate rows")
print("   ✅ Detected and removed outliers")
print("   ✅ Converted data types correctly")
print("   ✅ Created clean, model-ready data!")



6. Cleaning Summary
ملخص التنظيف

📊 Original rows: 22
الصفوف الأصلية: 22
✅ Final cleaned rows: 18
الصفوف النهائية بعد التنظيف: 18

🗑️  Rows removed: 4 (18.2%)
الصفوف المحذوفة: 4

📄 Cleaned Data (first 10 rows):
البيانات النظيفة:
   id       name   age  salary department  experience_years
0   1   Person_1  25.0   50000         IT                 2
1   2   Person_2  30.0   60000         HR                 5
2   3   Person_3  40.5   55000         IT                 3
3   4   Person_4  35.0   70000    Finance                 8
4   5   Person_5  40.0   80000         IT                10
5   6   Person_6  40.5   65000         HR                 4
6   7   Person_7  28.0   58000         IT                 2
7   8   Person_8  32.0   72000    Finance                 6
8   9   Person_9  45.0   90000         IT                12
9  10  Person_10  50.0   95000         HR                15

✅ Example 2 Complete! ✓
اكتمل المثال 2! ✓

🎓 What you accomplished:
   ✅ Handled missing values (imputed with

In [25]:
# Convert age to int
df_clean['age'] = df_clean['age'].round().astype(int)
print("\nData types after conversion:")
print("أنواع البيانات بعد التحويل:")
print(df_clean.dtypes)


Data types after conversion:
أنواع البيانات بعد التحويل:
id                   int64
name                object
age                  int64
salary               int64
department          object
experience_years     int64
dtype: object


In [26]:
# 6. Summary
# الملخص
print("\n" + "=" * 60)
print("6. Cleaning Summary")
print("ملخص التنظيف")
print("=" * 60)
print(f"\nOriginal rows: {df.shape[0]}")
print(f"الصفوف الأصلية: {df.shape[0]}")
print(f"Final cleaned rows: {df_clean.shape[0]}")
print(f"الصفوف النهائية بعد التنظيف: {df_clean.shape[0]}")
print(f"\nRows removed: {df.shape[0] - df_clean.shape[0]}")
print(f"الصفوف المحذوفة: {df.shape[0] - df_clean.shape[0]}")
print("\nCleaned Data:")
print("البيانات النظيفة:")
print(df_clean.head(10))
print("\n" + "=" * 60)
print("Example 2 Complete! ✓")
print("اكتمل المثال 2! ✓")
print("=" * 60)


6. Cleaning Summary
ملخص التنظيف

Original rows: 22
الصفوف الأصلية: 22
Final cleaned rows: 18
الصفوف النهائية بعد التنظيف: 18

Rows removed: 4
الصفوف المحذوفة: 4

Cleaned Data:
البيانات النظيفة:
   id       name  age  salary department  experience_years
0   1   Person_1   25   50000         IT                 2
1   2   Person_2   30   60000         HR                 5
2   3   Person_3   40   55000         IT                 3
3   4   Person_4   35   70000    Finance                 8
4   5   Person_5   40   80000         IT                10
5   6   Person_6   40   65000         HR                 4
6   7   Person_7   28   58000         IT                 2
7   8   Person_8   32   72000    Finance                 6
8   9   Person_9   45   90000         IT                12
9  10  Person_10   50   95000         HR                15

Example 2 Complete! ✓
اكتمل المثال 2! ✓
